In [6]:
pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 4.8 MB/s eta 0:00:00


In [3]:
import pandas as pd

# Replace 'your_file.csv' with the path to your CSV file
file_path = 'scraped_preprocessed_df.csv'

# Read the CSV file into a pandas DataFrame
data = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to verify that it was read correctly
print(data.head(1))

   Unnamed: 0           Title Company/Location  \
0           0  Data Scientist     commitremote   

                                         Description  
0  skillsdo experience web service yesno job deta...  


In [22]:
data.shape

(120, 4)

In [4]:
column_names = data.columns

# Print column names
print("Column names:")
for column in column_names:
    print(column)

Column names:
Unnamed: 0
Title
Company/Location
Description


In [18]:
from pinecone import Pinecone,  ServerlessSpec

pc = Pinecone(api_key='4f6218a4-19c8-44de-b9f9-cba78dd0e28b')

In [19]:
pc.create_index(
    name="description",
    dimension=300, # Replace with your model dimensions
    metric="euclidean", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-west-2"
    )
)

In [20]:
index = pc.Index("description")

In [1]:
import gensim.downloader as api
import numpy as np

# Load pre-trained Word2Vec model
word2vec_model = api.load("word2vec-google-news-300")


# Save the model to a file
model_file_path = "word2vec_model.bin"
word2vec_model.save(model_file_path)

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [25]:
def text_to_vector(text):
    if isinstance(text, str):
        # Split the text into words
        words = text.split()
        # Initialize an empty vector
        vector = np.zeros(word2vec_model.vector_size)
        # For each word in the text, get its word vector and add it to the vector
        for word in words:
            if word in word2vec_model:
                vector += word2vec_model[word]
        # Normalize the vector
        if words:
            vector /= len(words)
    else:
        # If the text is not a string, return a vector of zeros
        vector = None
    return vector

vectors = []
# Convert each description into a vector and upsert it into the index
for idx, description in enumerate(data["Description"]):

    # Convert description to vector
    description_vector = text_to_vector(description)
    if description_vector is not None:
    # Construct the vector object with metadata
      vector_object = {
          "id": str(idx),
          "values": description_vector.tolist(),  # Convert numpy array to list
        }

      vectors.append(vector_object)
    # Upsert vector into the index
index.upsert(vectors)

{'upserted_count': 117}

In [26]:
file_path = 'resume_preprocessed_df.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to verify that it was read correctly
print(df.head(1))

   Unnamed: 0      Category                                             Resume
0           0  Data Science  skill programming language python panda numpy ...


In [46]:
resume_vector = text_to_vector(df['Resume'][5])

if resume_vector is not None:
    # Convert the resume vector from NumPy array to a Python list
    resume_vector_list = resume_vector.tolist()
    # Query using the converted resume vector list

    results = index.query(vector=resume_vector_list, top_k=3, include_values=False)
    print(results)

{'matches': [{'id': '48', 'score': 0.352579117, 'values': []},
             {'id': '42', 'score': 0.352579117, 'values': []},
             {'id': '87', 'score': 0.396939099, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}


In [47]:
# Assuming 'results' contains the output you provided
matches = results['matches']

# Extract IDs from matches
ids = [match['id'] for match in matches]

# Retrieve titles from 'data' DataFrame based on IDs
titles = [data.loc[int(match_id)]['Title'] for match_id in ids]

# Print the titles
print("Titles:")
for title in titles:
    print(title)


Titles:
Data Scientist
Data Scientist
IT Support Service Analyst (M-F 8AM-4PM & M-F 9:30AM-5:30PM)
